In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/mcstllns/DeepLearning/main/phone_price_classification_train.csv'
df = pd.read_csv(url)
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [4]:
price_range_counts = df['price_range'].value_counts()
print(price_range_counts)


price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64


In [23]:
# Supress info messages
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# Import libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sys
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

import pandas as pd
import time

print(tf.__version__)

2.13.0


In [6]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
  
  plt.figure(figsize=(3,2))
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error')
  plt.plot(hist['epoch'], hist['loss'],'r--',
           label='Training Error')
  plt.plot(hist['epoch'], hist['accuracy'],'b',
           label = 'accuracy')
  plt.legend()
  plt.tight_layout()
  plt.show() 


In [7]:
x_train = df.sample(frac=0.8, random_state=0)
x_test = df.drop(x_train.index)

y_train = x_train.pop('price_range')
y_test = x_test.pop('price_range')

In [25]:
# Guardamos las dimensiones
ntrain = x_train.shape[0]
ntest  = x_test.shape[0]
dimf = 19 #x_train.shape[1]
dimc = 19 #x_train.shape[1]

print(ntrain, ntest, dimf, dimf)
# preproceso
x_train = x_train.astype('float32')/255.
# x_train = x_train.reshape(ntrain, dimf*dimc)
x_test = x_test.astype('float32')/255.
# x_test = x_test.reshape(ntest, dimf*dimc)

1600 400 19 19


In [26]:
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [27]:
model = keras.Sequential()
model.add(Dense(30, activation='sigmoid', input_shape=(dimf*dimc,)))
model.add(Dense(4, activation = 'softmax'))

In [28]:
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.2, beta_1=0.1),
    metrics = ['accuracy'])

In [29]:
history = model.fit(y_train, y_test,
              # batch_size=100,           # proporcional al conjunto de datos
              epochs=30,
              # validation_split = 0.2,   # partimos en train y dev
              verbose=0)                # para que no ocupe toda la pantalla

ValueError: Data cardinality is ambiguous:
  x sizes: 1600
  y sizes: 400
Make sure all arrays contain the same number of samples.